In [ ]:
import bs4
import pandas as pd
import time,re
from selenium.webdriver.common.action_chains import ActionChains
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

# 드라이버 위치 설정
driver_loc = "./chromedriver.exe"
# 드라이버 옵션 설정

options = webdriver.ChromeOptions()

options.add_argument("window-size=2580x1440") # 파이썬이 크롬창을 띄울 것인데 창 크기 선택

# 웹 드라이버 정의
driver = webdriver.Chrome(driver_loc, options=options)

# 에누리 웹페이지 이동
# URL 정의
smartwatchUrl = 'http://www.enuri.com/'



driver.maximize_window()

driver.get(smartwatchUrl)


time.sleep(1.5)
# driver.switch_to_default_content # 처음 iframe

tag = driver.find_element_by_xpath('//*[@id="gnbMenu"]/li[4]/a/em')
tag.click()

tag_main =driver.find_element_by_xpath('//*[@id="first_depth3"]/li[7]/p') 
tag_sub = driver.find_element_by_xpath('//*[@id="first_depth3"]/li[7]/div/ul[2]/li[1]/a')

ActionChains(driver).move_to_element(tag_main).click(tag_sub).perform()
time.sleep(1)

# 가격비교 이동

compareXpath = '//*[@id="tabsDiv"]/li[2]/a'
compareMenu = driver.find_element_by_xpath(compareXpath)
compareMenu.click()
time.sleep(0.5)

# 90개씩 보기
viewXpath = '//*[@id="pageGapSelDiv"]/a'
viewMenu = driver.find_element_by_xpath(viewXpath)
viewMenu.click()
time.sleep(0.5)

ninetyXpath = '//*[@id="pageGapSelDiv"]/ul/li[3]/a'
ninetyMenu = driver.find_element_by_xpath(ninetyXpath)
ninetyMenu.click()
time.sleep(0.5)

reviewList = []


for i in range(0, 10):
    
    
#     WebDriverWait(driver, 20).until(EC.presence_of_element_located((By.CLASS_NAME, "detailMultiLink prodName")))
#     WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.CLASS_NAME, "btn next")))
      
    if i > 0:
            # 다음 페이지 이동
            url = 'http://www.enuri.com/list.jsp?cate=03181105&tabType=' + str(i + 1)
            driver.get(url)
            
    
    time.sleep(3)  
    data = driver.page_source
#     print(data)   

    bs = bs4.BeautifulSoup(data, 'html.parser')

    ulTag_1 = bs.find(name = 'ul',
                   attrs = {'id': 'listBodyDiv'})
    liTags_1 = ulTag_1.findAll(name = 'li',
                          attrs = {'class': 'prodItem wide'})
    print(len(liTags_1))
    
    
   
    # xpath이용 -> //*[@id="xxxx"]/div[1]/strong/a ,xxxx부분에 상품 id
#     if (None == re.search('prodItem wide*', )) & (len(commentList) > 5):

    for j in range(len(liTags_1)):
        productNum = liTags_1[j].attrs['id']
        print(j, productNum,type(productNum))
        productXpath = '//*[@id="{}"]/div[1]/strong/a'.format(productNum)
        driver.find_element_by_xpath(productXpath).click()
        time.sleep(2)

        second_tab = driver.window_handles[1]
        driver.switch_to.window(second_tab)

        # 상품평으로 이동
        
        itemReviewXpath = '//*[@id="tab3"]/a'
        itemReviewMenu = driver.find_element_by_xpath(itemReviewXpath)
        itemReviewMenu.click()
        time.sleep(1.5)

        data3 = driver.page_source
        reviewBs = bs4.BeautifulSoup(data3, 'html.parser')

        divTag_2 = reviewBs.find('div', {'class':'commentbox'})
        ulTag_2 = divTag_2.find('ul', {'class':'comment__list'})
        liTags_2 = ulTag_2.findAll('li')
        columnList = []

        for k in range(len(liTags_2)):
            review = liTags_2[k].find('p', {'class':'sub bbs_click_class'})
            columnList.append(review.text.strip())
        reviewList.append(columnList)
        
        df_reviewList = pd.DataFrame(reviewList)
        print(df_reviewList)
#         df_reviewList.to_csv('./{}.csv'.format(productNum), index = False, encoding ='ms949')

        driver.close()
        first_tab = driver.window_handles[0]
        driver.switch_to.window(first_tab)


    